In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import torchvision.models as models
import numpy as np
from copy import deepcopy

import matplotlib.pyplot as plt
from random import randint
from torchvision.datasets import MNIST

import time

import os
!ls
os.chdir("/content/drive/MyDrive/Saravanan/")
from datasets import return_dataset
[[mnist_train, mnist_test], [fashionmnist_train, fashionmnist_test], [cifar10_train, cifar10_test]] = return_dataset()
datasets = [[mnist_train, mnist_test], [fashionmnist_train, fashionmnist_test], [cifar10_train, cifar10_test]]

from initializations import initialize, negative_model, distorted_model



concatenated.png	     __pycache__
data			     resnet_training_script_adam.ipynb
datasets.py		     temp3.jpg
datasets_vggnet.py	     temp4.jpg
Features		     training_data_adam
Features_Extraction.ipynb    training_data_resnet_adam
initializations.py	     training_data_sgd
lenet.py		     training_data_vggnet
lenet_training_script.ipynb  vggnet_training_adam.ipynb
MNIST			     vggnet_training_script_sgd.ipynb
MNIST.tar.gz.1		     vggnet_training_sgd.ipynb
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
#get data
from datasets import return_dataset
[[mnist_train, mnist_test], [fashionmnist_train, fashionmnist_test], [cifar10_train, cifar10_test]] = return_dataset()
datasets = [[mnist_train, mnist_test], [fashionmnist_train, fashionmnist_test], [cifar10_train, cifar10_test]]

from initializations import initialize, negative_model, distorted_model



Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def run_train(model, traindata, testdata, 
              lr=0.005, momemntum=0.9, epochsnum=20, batch_size=128, num_workers=2,
              rootadd = "/content/drive/MyDrive/Saravanan/vgg_cifar/",
              save_epochs=[]):

  name = model.name + '-init=' + model.initname + "-on-" + traindata.name + '-' + str(epochsnum) + "epochs" + "-" + model.optimizer
  folderadd = rootadd + name
  
  
  # check folder does not exsit
  trynumber = 2
  testadd = folderadd
  while path.exists(testadd):
    testadd = folderadd + '_try' + str(trynumber)
    trynumber += 1
  folderadd = testadd + "/"
  mkdir(folderadd)

  initadd = folderadd + "initial_model.pt"
  torch.save(model, initadd)



  trainloader = torch.utils.data.DataLoader(traindata, batch_size=batch_size,
                                          shuffle=True, num_workers=num_workers)
  testloader = torch.utils.data.DataLoader(testdata, batch_size=batch_size,
                                         shuffle=False, num_workers=num_workers)
  dataloaders_dict = {'train': trainloader, 'val': testloader}
  
  criterion = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momemntum)
  

  model.to(device)
  print('\n' * 3)
  print(name + '\n' + '='*len(name))
  model, test_history, train_history = train_model(model=model, dataloaders=dataloaders_dict,
                    criterion=criterion, optimizer=optimizer, num_epochs=epochsnum,
                    folder=folderadd, save_epochs=save_epochs)

  model.traindata = traindata.name
  model.train_history = train_history
  model.test_history = test_history
  model.epochs_trained = epochsnum  

  # save model
  modeladd = folderadd[:-1] + ".pt"
  torch.save(model, modeladd)
  return model
'done'

'done'

In [ ]:
# train models
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

initlist = [ 'kaiming_uniform', 'kaiming_normal']
# datasets = [[mnist_train, mnist_test], [fashionmnist_train, fashionmnist_test], [cifar10_train, cifar10_test]]

epochsnum = 20
lr = 0.005
momemntum=0.9
batch_size=512
num_workers=2
save_epochs = [x for x in range(epochsnum) if x%5 == 0]

features_size = 512
classes_size = 10


for initname in initlist:
    if initname == 'pre_trained':
      model = models.vgg19(pretrained=True)
      model.name = 'vgg19'
      model.optimizer = 'sgd'
      model.fc = torch.nn.Linear(features_size, classes_size, bias=True)
      model.initname = initname
    else:
      model = models.vgg19(pretrained=False)
      model.name = 'vgg19'
      model.optimizer = 'sgd'
      model.fc = torch.nn.Linear(features_size, classes_size, bias=True)

      initialize(model, initname)

    for data in datasets:

      finalmodel = run_train(model=model, traindata=data[0], testdata=data[1],
                epochsnum=epochsnum, save_epochs=save_epochs)
      if initlist=='pretrained':
        pass
      else:
        negativemodel = negative_model(finalmodel)
        run_train(model=negativemodel, traindata=data[0], testdata=data[1],
                epochsnum=epochsnum, save_epochs=save_epochs)
      
      distortedmodel = distorted_model(finalmodel)
      run_train(model=distortedmodel, traindata=data[0], testdata=data[1],
                epochsnum=epochsnum, save_epochs=save_epochs)






vgg19-init=kaiming_uniform-on-cifar_trn-2epochs-sgd
Epoch 0/1
----------
train Loss: 2.2778 Acc: 0.1467
val Loss: 1.8620 Acc: 0.2413

saved epoch 0 on g19-init=kaiming_uniform-on-cifar_trn-2epochs-sgd/epoch_0.pt
Epoch 1/1
----------
train Loss: 1.7076 Acc: 0.3281
val Loss: 1.5234 Acc: 0.4179

Training complete in 1m 50s
Best val Acc: 0.417900




vgg19-init=negative_kaiming_uniform-on-cifar_trn-2epochs-sgd
Epoch 0/1
----------
train Loss: 2.8431 Acc: 0.0985
val Loss: 2.3375 Acc: 0.1000

saved epoch 0 on negative_kaiming_uniform-on-cifar_trn-2epochs-sgd/epoch_0.pt
Epoch 1/1
----------
train Loss: 2.3352 Acc: 0.1009
val Loss: 2.3132 Acc: 0.1000

Training complete in 1m 48s
Best val Acc: 0.100000




vgg19-init=distorted_kaiming_uniform-on-cifar_trn-2epochs-sgd
Epoch 0/1
----------
train Loss: 1.3880 Acc: 0.4882
val Loss: 1.2154 Acc: 0.5627

saved epoch 0 on istorted_kaiming_uniform-on-cifar_trn-2epochs-sgd/epoch_0.pt
Epoch 1/1
----------
train Loss: 1.1450 Acc: 0.5893
val Loss: 1.075